In [1]:
dbutils.notebook.run("secrets", 30)
MOUNT_NAME = "optcad"

In [2]:
import pandas as pd
from pprint import pprint
from pyspark import  SparkContext
from pyspark import SparkConf
from pyspark.sql import DataFrame
from IPython.display import display, HTML
import matplotlib.pyplot as plt

deals_df = spark.read.csv("/mnt/%s/static/testing/deals.csv" % MOUNT_NAME, header=True)
metarisks_df = spark.read.csv("/mnt/%s/static/testing/metarisks.csv" % MOUNT_NAME, header = True)
losses_df = spark.read.parquet("/mnt/%s/calculator/*/parquet/*.parquet" % MOUNT_NAME)

metarisks_df.createOrReplaceTempView("metarisks")
losses_df.createOrReplaceTempView("losses")
deals_df.createOrReplaceTempView("deals")

deals_df.toPandas()

In [3]:
allocs_df = deals_df[[
    u'DealReference',
    u'CCF Cronus',
    u'ECF Emerald',
    u'FCF Ferrous',
    u'HCF Hudson',
    u'ICF Iron',
    u'JCF Juniper',
    u'KCT Kumo Catastrophe Trust',
    u'LFL Lotus Master Fund Ltd',
    u'NCF Nephila',
    u'NSHF Nephila Syndicate Holdings',
    u'OCF Onyx',
    u'PCF Palmetto',
    u'RBK Rubik',
    u'RCF Rembrandt',
    u'TFL Triton',
    u'TYC Tyche',
    u'XCF Xanth',
    u'YCF Yderst',
    u'ALL'
]]
allocs_df = pd.melt(allocs_df.toPandas(), id_vars="DealReference", var_name="Fund", value_name="Allocation").sort_values("DealReference")

allocs_df = sqlContext.createDataFrame(allocs_df)
allocs_df.createOrReplaceTempView("allocs")

In [4]:
%sql
SELECT DealReference, MetaRiskId, (SUM(LayerAttrition) / 10000) as CalculatedAggEL, ELAgg as ExpectedAggEL, 
ABS((SUM(LayerAttrition) / 10000) - ELAgg) as Delta, ELTID
FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId 
WHERE metarisks.MetaRiskId = 51
GROUP BY DealReference, MetaRiskId, ELAgg, ELTID
ORDER BY Delta DESC


In [5]:
%sql
CACHE TABLE deal_yelt AS

SELECT Simulation, DealReference, ELTID, EventId, EventDay,
SUM(deals.Limit * (CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) * losses.LayerAttrition) as MetaRiskGrossLoss,
SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) *(losses.LayerAttrition - deals.ROL * losses.ReinstatementAttrition))  as MetaRiskNetLoss,
SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN -1 ELSE 1 END) * deals.ROL * losses.ReinstatementAttrition) as MetaRiskReinstatementPremium, MetaRiskId 
FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId WHERE metarisks.MetaRiskId IN (1, 24, 28, 34, 46, 48, 49, 50, 51, 53)
GROUP BY Simulation, MetaRiskId, DealReference, ELTID, EventId, EventDay
ORDER By Simulation

In [6]:
%sql
--CACHE TABLE allocations_ylt AS

SELECT deal_yelt.MetaRiskId, deal_yelt.Simulation, allocs.Fund,
SUM(MetaRiskNetLoss * Allocation) as AggNet,
MIN(MetaRiskNetLoss * Allocation) as OccNet,
SUM(MetaRiskGrossLoss * Allocation) as AggGross,
MIN(MetaRiskGrossLoss * Allocation) as OccGross
FROM
deal_yelt
INNER JOIN
deals
ON deal_yelt.DealReference = deals.DealReference
AND deal_yelt.ELTID = deals.ELTID
INNER JOIN allocs 
ON allocs.DealReference = deal_yelt.DealReference
WHERE MetaRiskId IN (1, 24, 28, 34, 46, 48, 49, 50, 51, 53)
GROUP BY MetaRiskId, Simulation, allocs.Fund
ORDER By Simulation, MetaRiskId


In [7]:
%sql 
CREATE TEMPORARY TABLE eventsets
USING parquet
OPTIONS (
  path "/mnt/optcad/calculator/*/parquet/*.parquet"
)

In [8]:
%sql
SELECT count(distinct eventsetid)
FROM
eventsets

In [9]:
%python
df_fcf = spark.sql("SELECT MetaRiskId, Simulation, AggGrossFCF FROM allocations_ylt").toPandas()
df_fcf["Pctile"] = df_fcf.AggGrossFCF.rank()

df_sql = spark.createDataFrame(df_fcf).createOrReplaceTempView("scatter")


In [10]:
%sql
SELECT * FROM scatter